# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle
import os 
import glob
from skimage import io
from skimage import color, exposure, transform
import pandas as pd
import numpy as np


def get_label(file_dir,label):
    return file_dir[1][label][-2::]

def Image_transform(img,x1,y1,x2,y2):
    # Histogram normalization in v channel
    #hsv = color.rgb2hsv(img)
    #hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    #img = color.hsv2rgb(hsv)

    # Focus on Region of interest 
    img = img[y1: y2, x1: x2, :]
    
    # rescale to standard size
    size = 32
    img = transform.resize(img, (size, size))

    # roll color axis to axis 0
    img = np.rollaxis(img, -1)

    return img    

# Load training data

file_dir_name = 'GTSRB_Final_Training_Images/GTSRB/Final_Training/Images'

all_img_paths = glob.glob(os.path.join(file_dir_name, '*/*.ppm'))
np.random.shuffle(all_img_paths)

train_number = int(0.7*len(all_img_paths))
train_img_paths = all_img_paths[:train_number]
validation_img_paths = all_img_paths[train_number + 1 :]

train_imgs = []
train_labels = []
ROI = []

for img_path in train_img_paths:
    ROI = []
    data = pd.read_csv(file_dir_name + '/' + img_path.split('\\')[1] + '/GT-' + img_path.split('\\')[1] + '.csv')
    print(img_path.split('\\')[1])
    print(img_path.split('\\')[2])
    for i in range(int(data.shape[0])):
        temp = str(data.at[i,"Filename;Width;Height;Roi.X1;Roi.Y1;Roi.X2;Roi.Y2;ClassId"])
        if img_path.split('\\')[2] == temp.split(";")[0]:
            print('found')
            label = int(temp.split(";")[7])
            ROI.extend((temp.split(";")[3],temp.split(";")[4],temp.split(";")[5],temp.split(";")[6]))
            print(ROI)
            break;
    img = io.imread(img_path)
    train_imgs.append(img)
    train_labels.append(label)
    
# Load validation data
    
validation_imgs = []
validation_labels = []
ROI = []

for img_path in validation_img_paths:
    ROI = []
    data = pd.read_csv(file_dir_name + '/' + img_path.split('\\')[1] + '/GT-' + img_path.split('\\')[1] + '.csv')
    print(img_path.split('\\')[1])
    print(img_path.split('\\')[2])
    for i in range(int(data.shape[0])):
        temp = str(data.at[i,"Filename;Width;Height;Roi.X1;Roi.Y1;Roi.X2;Roi.Y2;ClassId"])
        if img_path.split('\\')[2] == temp.split(";")[0]:
            print('found')
            label = int(temp.split(";")[7])
            ROI.extend((temp.split(";")[3],temp.split(";")[4],temp.split(";")[5],temp.split(";")[6]))
            print(ROI)
            break;
    img = io.imread(img_path)
    validation_imgs.append(img)
    validation_labels.append(label)
    
# Load testing data

test_imgs = []
ROI = []

test_dir_name = 'GTSRB_Final_Test_Images\GTSRB\Final_Test'
test_img_paths = glob.glob(os.path.join(test_dir_name, '*/*.ppm'))
data = pd.read_csv(test_dir_name +  '/Images/GT-final_test.test.csv')
ppm_count = 0
for img_path in test_img_paths:
    ROI = []
    temp = str(data.at[ppm_count,"Filename;Width;Height;Roi.X1;Roi.Y1;Roi.X2;Roi.Y2"])
    ROI.extend((temp.split(";")[3],temp.split(";")[4],temp.split(";")[5],temp.split(";")[6]))
    print(ROI)
    img = io.imread(img_path)
    test_imgs.append(img)
    ppm_count += 1
    
# Load label csv 
label = pd.read_csv('signnames.csv')
label_count = label.shape[0]

"""training_file = ?
validation_file=?
testing_file = ?

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']"""

00029
00004_00005.ppm
found
['5', '5', '33', '33']
00004
00038_00018.ppm
found
['6', '5', '41', '40']
00033
00004_00024.ppm
found
['6', '6', '60', '62']
00022
00009_00000.ppm
found
['5', '6', '27', '24']
00022
00008_00016.ppm
found
['6', '5', '37', '35']
00033
00011_00001.ppm
found
['5', '6', '24', '25']
00019
00006_00024.ppm
found
['7', '6', '78', '69']
00038
00010_00008.ppm
found
['5', '6', '25', '26']
00007
00029_00000.ppm
found
['6', '5', '23', '21']
00004
00057_00029.ppm
found
['11', '11', '111', '112']
00014
00000_00027.ppm
found
['12', '13', '132', '131']
00021
00002_00027.ppm
found
['7', '5', '66', '56']
00005
00007_00003.ppm
found
['5', '5', '25', '25']
00011
00029_00004.ppm
found
['6', '6', '49', '44']
00002
00037_00012.ppm
found
['6', '6', '31', '31']
00014
00025_00019.ppm
found
['5', '6', '41', '43']
00014
00008_00028.ppm
found
['7', '10', '79', '108']
00031
00014_00020.ppm
found
['6', '5', '64', '57']
00012
00052_00023.ppm
found
['12', '12', '127', '129']
00014
00024_00020

00028
00011_00000.ppm
found
['6', '5', '32', '29']
00001
00070_00004.ppm
found
['5', '6', '44', '50']
00024
00005_00026.ppm
found
['6', '5', '64', '59']
00005
00047_00010.ppm
found
['5', '6', '29', '30']
00030
00006_00028.ppm
found
['7', '6', '67', '66']
00009
00019_00018.ppm
found
['5', '5', '33', '34']
00035
00033_00006.ppm
found
['5', '6', '23', '25']
00013
00038_00015.ppm
found
['5', '5', '51', '46']
00012
00064_00019.ppm
found
['5', '5', '44', '47']
00020
00004_00023.ppm
found
['5', '5', '37', '37']
00013
00049_00020.ppm
found
['5', '6', '55', '50']
00026
00008_00013.ppm
found
['5', '5', '39', '35']
00040
00008_00000.ppm
found
['5', '5', '20', '24']
00004
00001_00018.ppm
found
['5', '6', '46', '47']
00004
00030_00018.ppm
found
['6', '5', '43', '42']
00033
00011_00027.ppm
found
['7', '7', '80', '80']
00002
00037_00008.ppm
found
['5', '6', '27', '28']
00004
00017_00014.ppm
found
['6', '5', '31', '32']
00008
00000_00026.ppm
found
['5', '5', '46', '50']
00003
00021_00000.ppm
found
['5

00029
00002_00012.ppm
found
['6', '5', '42', '38']
00016
00006_00003.ppm
found
['7', '6', '68', '67']
00008
00005_00004.ppm
found
['5', '5', '26', '28']
00016
00010_00029.ppm
found
['7', '7', '65', '67']
00004
00002_00005.ppm
found
['5', '5', '28', '28']
00002
00025_00002.ppm
found
['6', '5', '23', '22']
00000
00000_00016.ppm
found
['6', '5', '41', '39']
00007
00000_00012.ppm
found
['5', '6', '29', '30']
00029
00003_00014.ppm
found
['6', '6', '52', '48']
00007
00018_00012.ppm
found
['5', '5', '28', '29']
00008
00032_00023.ppm
found
['5', '5', '41', '42']
00004
00031_00020.ppm
found
['6', '5', '34', '34']
00013
00066_00006.ppm
found
['6', '6', '32', '29']
00004
00042_00001.ppm
found
['5', '5', '21', '21']
00011
00012_00019.ppm
found
['6', '6', '64', '60']
00011
00017_00028.ppm
found
['11', '10', '115', '102']
00002
00073_00012.ppm
found
['6', '6', '31', '31']
00004
00044_00013.ppm
found
['6', '5', '31', '32']
00027
00006_00007.ppm
found
['6', '5', '40', '35']
00003
00009_00026.ppm
found

00017
00022_00023.ppm
found
['5', '5', '59', '59']
00008
00016_00021.ppm
found
['5', '5', '41', '41']
00035
00003_00027.ppm
found
['5', '6', '39', '46']
00023
00010_00014.ppm
found
['5', '6', '49', '45']
00008
00010_00014.ppm
found
['6', '5', '34', '33']
00012
00055_00026.ppm
found
['8', '7', '77', '76']
00040
00003_00012.ppm
found
['5', '5', '44', '43']
00028
00017_00021.ppm
found
['6', '6', '63', '58']
00007
00005_00020.ppm
found
['5', '5', '55', '55']
00010
00049_00023.ppm
found
['5', '6', '42', '43']
00013
00028_00005.ppm
found
['5', '6', '42', '41']
00028
00017_00005.ppm
found
['5', '5', '32', '30']
00014
00003_00008.ppm
found
['5', '6', '44', '46']
00038
00032_00001.ppm
found
['5', '5', '27', '27']
00038
00005_00029.ppm
found
['7', '8', '71', '78']
00013
00054_00014.ppm
found
['8', '7', '84', '79']
00013
00031_00019.ppm
found
['5', '6', '36', '34']
00002
00042_00008.ppm
found
['6', '6', '24', '25']
00033
00018_00024.ppm
found
['6', '7', '40', '66']
00001
00052_00013.ppm
found
['6

00008
00019_00003.ppm
found
['6', '5', '29', '28']
00012
00017_00014.ppm
found
['5', '6', '40', '40']
00030
00006_00007.ppm
found
['5', '6', '25', '26']
00021
00009_00029.ppm
found
['11', '11', '124', '116']
00005
00057_00001.ppm
found
['5', '6', '25', '27']
00004
00016_00025.ppm
found
['5', '5', '45', '49']
00012
00032_00017.ppm
found
['5', '6', '39', '48']
00038
00000_00027.ppm
found
['7', '8', '77', '80']
00017
00011_00014.ppm
found
['5', '6', '23', '26']
00002
00057_00025.ppm
found
['5', '6', '58', '58']
00004
00014_00003.ppm
found
['5', '6', '23', '25']
00033
00003_00029.ppm
found
['9', '10', '100', '102']
00006
00010_00010.ppm
found
['6', '6', '30', '32']
00039
00004_00021.ppm
found
['5', '6', '51', '54']
00014
00018_00021.ppm
found
['8', '8', '81', '82']
00014
00020_00002.ppm
found
['5', '5', '36', '40']
00026
00013_00008.ppm
found
['6', '5', '30', '28']
00002
00046_00010.ppm
found
['6', '6', '31', '32']
00011
00033_00001.ppm
found
['6', '5', '29', '27']
00012
00048_00007.ppm
fo

...

00014
00012_00008.ppm
found
['5', '6', '39', '40']
00018
00014_00003.ppm
found
['6', '6', '33', '33']
00003
00044_00003.ppm
found
['6', '6', '24', '25']
00007
00040_00027.ppm
found
['6', '6', '38', '38']
00031
00019_00013.ppm
found
['5', '6', '48', '45']
00025
00048_00008.ppm
found
['5', '5', '30', '36']
00004
00056_00028.ppm
found
['6', '6', '49', '56']
00013
00039_00023.ppm
found
['5', '5', '53', '47']
00013
00051_00021.ppm
found
['6', '5', '55', '50']
00009
00000_00025.ppm
found
['6', '5', '47', '46']
00038
00064_00006.ppm
found
['6', '6', '28', '29']
00008
00009_00016.ppm
found
['5', '5', '34', '34']
00025
00021_00012.ppm
found
['6', '6', '38', '36']
00002
00038_00022.ppm
found
['7', '6', '71', '69']
00028
00016_00013.ppm
found
['5', '5', '40', '42']
00012
00018_00019.ppm
found
['5', '5', '53', '53']
00001
00011_00027.ppm
found
['8', '7', '83', '81']
00038
00051_00027.ppm
found
['8', '8', '78', '80']
00017
00015_00016.ppm
found
['6', '6', '34', '39']
00018
00006_00004.ppm
found
['6

00003
00031_00028.ppm
found
['8', '8', '78', '87']
00018
00036_00018.ppm
found
['6', '6', '62', '63']
00001
00003_00027.ppm
found
['10', '10', '104', '106']
00038
00035_00027.ppm
found
['6', '7', '69', '72']
00025
00047_00025.ppm
found
['14', '12', '145', '125']
00018
00014_00009.ppm
found
['6', '5', '37', '36']
00032
00006_00015.ppm
found
['6', '5', '33', '32']
00001
00063_00004.ppm
found
['6', '6', '22', '23']
00009
00016_00027.ppm
found
['6', '7', '61', '63']
00033
00014_00012.ppm
found
['5', '6', '33', '35']
00013
00032_00011.ppm
found
['5', '6', '42', '34']
00004
00003_00020.ppm
found
['5', '6', '38', '38']
00002
00007_00003.ppm
found
['5', '5', '30', '29']
00001
00023_00005.ppm
found
['5', '5', '31', '29']
00037
00005_00002.ppm
found
['5', '5', '32', '33']
00026
00008_00022.ppm
found
['6', '5', '59', '52']
00010
00018_00013.ppm
found
['5', '6', '35', '36']
00013
00017_00023.ppm
found
['8', '6', '76', '68']
00013
00028_00008.ppm
found
['5', '5', '43', '41']
00008
00033_00016.ppm
f

found
['6', '6', '31', '32']
00008
00030_00010.ppm
found
['6', '5', '25', '24']
00001
00027_00019.ppm
found
['5', '6', '43', '45']
00041
00007_00029.ppm
found
['7', '7', '64', '67']
00001
00038_00011.ppm
found
['5', '6', '27', '29']
00002
00043_00012.ppm
found
['6', '5', '27', '26']
00013
00008_00013.ppm
found
['5', '6', '43', '54']
00033
00013_00023.ppm
found
['6', '6', '67', '66']
00002
00072_00027.ppm
found
['5', '5', '45', '48']
00004
00044_00024.ppm
found
['6', '5', '46', '46']
00012
00029_00004.ppm
found
['6', '6', '38', '36']
00001
00067_00018.ppm
found
['5', '6', '45', '46']
00001
00014_00020.ppm
found
['5', '5', '40', '41']
00017
00002_00009.ppm
found
['5', '6', '22', '39']
00014
00003_00023.ppm
found
['9', '8', '91', '90']
00013
00010_00021.ppm
found
['6', '6', '62', '57']
00004
00037_00012.ppm
found
['5', '6', '32', '33']
00016
00011_00008.ppm
found
['6', '6', '24', '25']
00012
00046_00001.ppm
found
['5', '5', '23', '25']
00004
00007_00003.ppm
found
['5', '5', '25', '25']
00

00009
00020_00027.ppm
found
['6', '6', '25', '27']
00005
00038_00015.ppm
found
['5', '6', '31', '33']
00004
00025_00029.ppm
found
['10', '10', '113', '107']
00003
00030_00001.ppm
found
['5', '6', '20', '22']
00025
00043_00025.ppm
found
['8', '7', '83', '76']
00013
00013_00010.ppm
found
['5', '6', '43', '40']
00030
00003_00005.ppm
found
['6', '5', '42', '35']
00012
00002_00027.ppm
found
['11', '12', '115', '119']
00039
00003_00006.ppm
found
['6', '6', '26', '26']
00035
00014_00023.ppm
found
['5', '6', '51', '52']
00035
00027_00006.ppm
found
['6', '5', '27', '27']
00002
00001_00006.ppm
found
['5', '6', '23', '25']
00007
00015_00009.ppm
found
['6', '5', '34', '35']
00038
00051_00023.ppm
found
['6', '6', '59', '60']
00015
00020_00028.ppm
found
['7', '7', '77', '79']
00012
00022_00002.ppm
found
['5', '6', '29', '30']
00016
00010_00023.ppm
found
['6', '6', '47', '49']
00031
00012_00000.ppm
found
['5', '6', '26', '26']
00004
00039_00006.ppm
found
['5', '5', '28', '28']
00038
00062_00023.ppm
f

"training_file = ?\nvalidation_file=?\ntesting_file = ?\n\nwith open(training_file, mode='rb') as f:\n    train = pickle.load(f)\nwith open(validation_file, mode='rb') as f:\n    valid = pickle.load(f)\nwith open(testing_file, mode='rb') as f:\n    test = pickle.load(f)\n    \nX_train, y_train = train['features'], train['labels']\nX_valid, y_valid = valid['features'], valid['labels']\nX_test, y_test = test['features'], test['labels']"

In [39]:
print(img.shape)
print(len(all_img_paths))
train_img_path = all_img_paths[:int(0.7*len(all_img_paths))]
print(len(train_img_path))
test_dir = 'GTSRB_Final_Test_Images\GTSRB\Final_Test'

label = pd.read_csv('signnames.csv')
label_count = label.shape[0]
print(label.shape[0])
print(type(label.shape[0]))
test_dir_name = 'GTSRB_Final_Test_Images\GTSRB\Final_Test'
test_img_path = glob.glob(os.path.join(test_dir_name, '*/*.ppm'))
print(test_img_path[30])
data = pd.read_csv(test_dir_name +  '/Images/GT-final_test.test.csv')
print(data.at[0,"Filename;Width;Height;Roi.X1;Roi.Y1;Roi.X2;Roi.Y2"])
print(test_img_path[20].split('\\')[-1])

(39, 38, 3)
39209
27446
43
<class 'int'>
GTSRB_Final_Test_Images\GTSRB\Final_Test\Images\00030.ppm
00000.ppm;53;54;6;5;48;49
00020.ppm


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(train_imgs)

# TODO: Number of validation examples
n_validation = len(validation_imgs)

# TODO: Number of testing examples.
n_test = len(test_imgs)

# TODO: What's the shape of an traffic sign image?
image_shape = img.shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = label.shape[0]

print("Number of training examples =", n_train)
print("Number of validation examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.

%matplotlib inline

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")